In [2]:
# qiskit circuit & machine
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import qiskit_aer
from qiskit_ibm_runtime.fake_provider import FakeVigo

# widget-related
import anywidget
import traitlets

# host setting for dev mode
_SS_ = "http://localhost:5175/"

from machine_data_prep import getMachineMetaInfo
import json

In [3]:
class CircuitWriter(anywidget.AnyWidget):
    _esm = _SS_ + "index.js?anywidget"
    machine = traitlets.Unicode().tag(sync=True)
    circ_code = traitlets.Unicode().tag(sync=True)
    mode = traitlets.Unicode("write").tag(sync=True)

def getCircuitWriter(backend=None):
    machine_json = ""
    if backend is not None:
        machine = getMachineMetaInfo(backend)
        machine_json = json.dumps(machine)
    circ_code = ""
    return CircuitWriter(circ_code=circ_code, machine=machine_json)

In [4]:
w = getCircuitWriter()

In [5]:
w

CircuitWriter()

In [13]:
print(w.circ_code)

from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import qiskit_aer
from qiskit_ibm_runtime.fake_provider import FakeVigo
from revkit import netlist, oracle_synth, truth_table
import revkit.export.qiskit
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import EstimatorV2 as Estimator

circuit = QuantumCircuit(3, 3)

# Superposition
circuit.h(0)
circuit.h(1)

# Boolean Truth Table Function Oracle
tt_bool_1 = revkit.truth_table.from_expression("(ab)")
oracle_1 = revkit.oracle_synth(tt_bool_1)
oracle_gate_1 = oracle_1.to_qiskit().to_gate()
circuit.append(oracle_gate_1, [0, 1, 2])

# Pauli Observables
observables_labels = ["IXYII", "ZXYII"]
observables = [SparsePauliOp(label) for label in observables_labels]
mapped_observables = [observable.apply_layout(circuit.layout) for observable in observables]

# backend and optimizer
optimization_level = 1
backend = qiskit_aer.AerSimulator.from_backend(FakeVigo())
pa